In [4]:
!pip install beautifulsoup4
!pip install google
!pip install newspaper3k

In [3]:
import csv
from difflib import SequenceMatcher
import newspaper
from newspaper import Config
from newspaper import Article
import numpy as np
from os.path import exists
import pandas as pd

In [5]:
def removeLine(x):
    if x[0] =='|':
        return x[1:].strip()
    else:
        return x

In [ ]:
def similar(a, b): 
    return SequenceMatcher(None, a, b).ratio()

In [6]:
df = pd.read_csv("data/scraped_posts/scraped_reddit_posts.csv", usecols = ['url','title'])
list_of_urls= df['url'].values.tolist()
list_of_titles= df['title'].values.tolist()

In [7]:
df2 = df[df['url'].str.contains("web.archive.org|https://archive.")] 

In [8]:
df3 = df2[df2['title'].str.contains("|")]

In [9]:
df3[['teaser', 'answer']] = df3['title'].str.split('|',n=1, expand=True)

In [53]:
USER_AGENT = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0'

config = Config()
config.browser_user_agent = USER_AGENT
config.request_timeout = 10

In [54]:
list_of_urls= df3['url'].values.tolist()
list_of_teasers= df3['teaser'].values.tolist()

# modified from https://stackoverflow.com/questions/69711582/newspaper3k-export-to-csv-on-first-row-only
index=0
list_of_texts = []
for url in list_of_urls:
    # web.archive links work, but archive.fo or archive.is have time-out/captcha error
    # we can google the actual web page from the teaser title
    # replace archive links with first link in google search of teaser
    if 'https://archive.' in list_of_urls[index]:
        query = list_of_teasers[index]
        try:
            for j in search(query, tld="co.in", num=1, stop=1, pause=0):
                url = j
        except:
            pass #it appears there could be time out errors, can increase timeout
    try:
        article = newspaper.Article(url,config=config)
        article.download()
        article.parse()
        article_meta_data = article.meta_data
        text = article.text
        title = article.title
        # check that article title of webpage is at least 50% similar to the teaser
        # if not, it is probably the wrong article
        if similar(title,list_of_teasers[index]) < 0.5:
            text = None
            print(index, 'title did not match')
        else:
            print(index)
    except:
        text = None 
        print(index,'error')
    index+=1
    #Write to file [url, article text]
    list_of_texts.append(text)

0
1
2
3
4 title did not match
5
6
7
8
9
10
11 title did not match
12
13
14
15
16
17
18
19
20
21
22
23
24 title did not match
25
26 title did not match
27
28 title did not match
29
30
31
32
33
34
35 title did not match
36
37
38 title did not match
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63 title did not match
64
65 error
66
67
68
69
70
71
72 title did not match
73
74 title did not match
75 title did not match
76 title did not match
77 title did not match
78
79
80 title did not match
81
82
83
84 title did not match
85 title did not match
86 title did not match
87
88
89
90
91 title did not match
92
93
94
95
96 title did not match
97
98
99
100
101 error
102
103
104
105
106
107
108 title did not match
109
110
111
112 title did not match
113 title did not match
114
115 error
116
117
118
119
120


In [ ]:
df3['article']= list_of_texts

In [ ]:
df3['teaser'] = df3['teaser'].apply(lambda x: x.strip())


In [ ]:
df3 = df3[df3['title'].str.contains("\|")]

In [ ]:
df3['answer'] = df3['answer'].apply(lambda x: x.strip())

/var/folders/6v/7l4sg8g92sj0mn3b96s3bd740000gn/T/ipykernel_16976/3641060672.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['answer'] = df3['answer'].apply(lambda x: x.strip())


In [ ]:
df3 = df3.drop_duplicates(subset=['title'])

In [ ]:
df3 = df3[df3['article']!='']

In [ ]:
df3 = df3[df3['answer']!='']

In [ ]:
df3['answer'] = df3['answer'].apply(removeLine)

In [ ]:
df3.to_pickle('data_full_pandas.pkl')


In [ ]:
dnp = df3.to_numpy()
np.save('data_full_numpy.npy', dnp)

In [ ]:
df4 = df3[df3['title'].str.contains("\|")]